# 牛奶收集问题

## 目标和前提条件

在本示例中，您将了解如何利用数学优化来解决一个有运力限制的车辆路径规划问题：牛奶收集问题。只有一辆运力有限的油罐车，您需要确定油罐车每天收集一组农场牛奶的最佳路线。这是一个复杂的问题，但数学优化将帮助您找到解决方案！

本模型是 H. Paul Williams 所著《数学规划中的模型构建》第五版第 278-281 页和 336-337 页的例 23。

这个建模示例属于高级水平，我们假设您了解 Python 和 Gurobi Python API，并且具有构建数学优化模型的高级知识。通常，这些示例的目标函数和/或约束条件比较复杂，需要使用 Gurobi Python API 的高级功能。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

---
## 问题描述

一家小型牛奶加工公司需要从 20 个农场收集牛奶并运回仓库进行加工。公司有一辆运力为 80,000 升的油罐车。其中 11 个小型农场只需要每隔一天收集一次。其他 9 个农场需要每天收集。下表给出了各个农场相对于仓库（编号为 1）的位置以及它们的收集需求量。

![农场坐标](farmCoordinates.PNG)

目标是为油罐车找到每天的最优路线，同时需要考虑以下要求：
1. 访问所有"每天收集"的农场。
2. 访问部分"隔天收集"的农场。
3. 不超过运力限制。

在隔天，油罐车必须再次访问"每天收集"的农场，并访问前一天未访问的"隔天收集"农场。

---
## 模型构建

### 集合和下标
$i, j \in \text{Farms} = \{0,1,2, ..,20 \}$：农场的下标和集合。仓库的下标为 0。

$\text{everyDay} = \{0,1,2, ..,9 \} \subset \text{Farms}$：需要每天访问的农场。

$\text{otherDay} = \{10,11, 12, ..,20 \} \subset \text{Farms}$：需要隔天访问的农场。

$k \in K = \{1,2 \}$：隔天访问农场的日期类型。

$\text{Edges}= \{(i,j) \in Farms \times Farms \}$：允许的边集合。

$S_k \in S$：第 $k$ 天路线中的子回路。

$G = (\text{Farms} , \text{Edges})$：一个图，其中节点集为 $\text{Farms}$，边集为 $\text{Edges}$。

### 参数

$d_{i, j} \in \mathbb{R}^+$：农场 i 到农场 j 的距离，对于所有 $(i, j) \in \text{Edges}$。

注意，农场 i 到农场 j 的距离等于农场 j 到农场 i 的距离，即 $d_{i, j} = d_{j, i}$。

$C \in \mathbb{R}^+$：油罐车的运力。

$R_i \in \mathbb{R}^+$：农场 i 的牛奶收集需求量。

### 决策变量
$x_{i, j, k} \in \{0, 1\}$：如果第 k 天的路线直接连接农场 i 和农场 j，该变量等于 1。否则，该决策变量等于 0。

$y_{i, k} \in \{0, 1\}$：如果农场 $i \in otherDay$ 在第 k 天的路线中被访问，该变量等于 1。

### 目标函数
- **最短路线**。最小化两条路线的总距离。

\begin{equation}
\text{Min} \quad Z = \sum_{k \in K} \sum_{(i,j) \in \text{Edges}} \frac{1}{2} d_{i,j} \cdot x_{i,j,k}
\tag{0}
\end{equation}

### 约束条件
- **对称约束**。对于每条边 $(i,j)$，如果农场 i 在访问农场 j 之前或之后被立即访问，则确保农场 i 和 j 相连。

\begin{equation}
x_{i, j, k} = x_{j, i, k} \quad \forall k \in dayType, \; (i, j) \in Edges
\tag{1}
\end{equation}

- **进入和离开每天访问的农场**。对于每个农场 i，确保该农场与其他两个农场相连。

\begin{equation}
\sum_{j: (i,j) \in \text{Edges}} x_{i,j,k} = 2 \quad \forall  i \in everyDay, \; k \in dayType 
\tag{2}
\end{equation}

- **进入和离开隔天访问的农场**。对于每个农场 i，确保该农场与其他两个农场相连。

\begin{equation}
\sum_{j: (i,j) \in \text{Edges}} x_{i,j,k}  = 2 \cdot y_{i, k} \quad \forall  i \in otherDay, \; k \in dayType 
\tag{3}
\end{equation}

- **油罐车运力**。油罐车的运力有限。
\begin{equation}
\sum_{i \in \text{otherDay}} R_{i} \cdot y_{i,k} \leq C -\sum_{i \in everyDay} R_{i} \quad \forall  k \in K 
\tag{4}
\end{equation}

- **农场访问**。限制某些农场只能隔天访问。

\begin{equation}
y_{i,1} + y_{i,2}  = 1 \quad \forall  i \in \text{otherDay}
\tag{5}
\end{equation}

- **子回路消除**。这些约束确保每条路线都没有循环。

\begin{equation}
\sum_{(i,j) \in S_k}x_{i,j,k} \leq |S_k|-1 \quad \forall  k \in K, \;   S_k \in S
\tag{6}
\end{equation}

其中子集 S 是由路线访问的农场集合，这条路线由不等式左边决策变量的值定义。

## Python 实现

我们导入 Gurobi Python 模块和其他 Python 库。

In [ ]:
# %pip install gurobipy

In [17]:
import gurobipy as gp
from gurobipy import GRB

import math
from itertools import combinations

# tested with Python 3.7.0 & Gurobi 9.1.0

### 输入数据
我们定义模型的所有输入数据。

In [18]:
# 创建字典来存储农场坐标（10英里）和收集需求量（1,000）

farms, coordinates, collect  = gp.multidict({
    0: [(0,0),0],
    1: [(-3,3),5],
    2: [(1,11),4],
    3: [(4,7),3],
    4: [(-5,9),6],
    5: [(-5,-2),7],
    6: [(-4,-7),3],
    7: [(6,0),4],
    8: [(3,-6),6],
    9: [(-1,-3),5],
    10: [(0,-6),4],
    11: [(6,4),7], 
    12: [(2,5),3],
    13: [(-2,8),4],
    14: [(6,10),5],
    15: [(1,8),6],
    16: [(-3,1),8],
    17: [(-6,5),5],
    18: [(2,9),7],
    19: [(-6,-5),6],
    20: [(5,-4),6]
})

# 包含仓库的农场列表
farms = [*range(0,21)]

# 每天访问的农场列表
everyDay = [*range(0,10)]

# 隔天访问的农场列表
otherDay = [*range(10,21)]

# 日期类型列表
dayType = [1,2]

# 油罐车运力（1,000）
tankerCap = 80

# 每天访问农场的需求量
everyDayReq = 0
for i in everyDay:
    everyDayReq += collect[i]

### 数据处理
这里，我们计算每对农场之间的距离并存储在 Python 字典中。

In [19]:
# 计算成对距离矩阵
# numpy linalg norm = 欧几里得距离 n=2

def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(farms, 2)}

### 模型部署
现在我们通过定义决策变量、约束条件和目标函数来确定牛奶收集问题的运力限制车辆路线规划模型。

In [ ]:
# 创建模型对象 m
m = gp.Model('MilkCollection.lp')

# 决策变量：

# 边变量 = 1，如果农场'i'在第'k'天的路线上与农场'j'相邻
vars = m.addVars(dist, dayType, vtype=GRB.BINARY, name='x')

# 隔天变量 = 1，如果农场'i'在第'k'天的路线上被访问
other_var = m.addVars(otherDay, dayType, vtype=GRB.BINARY, name='y') 

# 对称约束：复制对象（不是约束）
vars.update({(j,i,k):vars[i,j,k] for i,j,k in vars.keys()})

# 每天访问约束：每天访问的农场在第'k'天的路线上与两条边相连
m.addConstrs((vars.sum(i,'*',k) == 2 for i in everyDay for k in dayType  ), name='everyDay')

# 隔天访问约束：隔天访问的农场在第'k'天的路线上与两条边相连
m.addConstrs((vars.sum(i,'*',k) == 2*other_var[i,k] for i in otherDay for k in dayType ), name='otherDay')

# 油罐车运力约束
m.addConstrs(( gp.quicksum(collect[i]*other_var[i,k] for i in otherDay ) <= tankerCap-everyDayReq for k in dayType ),
             name='tankerCap')

# 隔天农场在第1天或第2天被访问
otherDayFarms = m.addConstrs((other_var.sum(i, '*') == 1 for i in otherDay), name='visited')

# 避免对称替代解
other_var[11,1].lb = 1

# 目标函数：最小化总行驶距离
m.setObjective(gp.quicksum(dist[i,j]*vars[i,j,k] for i,j in dist.keys() for k in dayType), GRB.MINIMIZE)

### 查找循环
以下函数确定一个未连接到仓库的循环。

In [14]:
# 从解的值中找到每个日期类型的边，作为元组列表
def selected(vals):
    s = {k:gp.tuplelist() for k in dayType}
    for i, j, k in vals.keys():
        if vals[i,j,k] > 0.5:
            s[k].append((i,j))
    return s
# 另一种方法，使用推导式语法：
#    return {k:gp.tuplelist((i, j) for i, j, k in vals.keys().select('*','*',k) if vals[i,j,k] > 0.5) for k in dayType}
            
# 给定一个边的元组列表，找到最短的子回路
def subtour(edges):
    nodes = set(i for e in edges for i in e)
    unvisited = list(nodes)
    cycle = list(nodes)
    while unvisited:  
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle): # 即使它是相同的，我们也会重复使用它，这样我们就能按顺序完成最后的遍历
            cycle = thiscycle # 新最短子线路
    return cycle

### 回调函数定义
子回路约束防止一个路线中出现多个循环。由于这些约束的数量呈指数级增长，我们不希望将它们全部添加到模型中。相反，我们使用回调函数来查找违反子回路约束的情况，并将它们作为延迟约束添加到模型中。

In [15]:
# 回调函数 - 使用延迟约束消除子回路
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # 获取当前解中选择的边
        vals = model.cbGetSolution(model._vars)
        edges = selected(vals)
        for k in dayType:
            tour = subtour(edges[k])
            if len(tour) < 0.5*len(edges[k]): # 0.5是由于对称性：存在边i,j和j,i
                # 为当天访问的农场添加子回路消除约束
                model.cbLazy(gp.quicksum(model._vars[i, j, k] for i, j in combinations(tour, 2))
                             <= len(tour)-1)

## 求解
现在我们可以使用延迟子回路约束来优化模型。

In [ ]:
# 优化模型

m.reset()
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

## 分析

我们打印最优总行驶距离和每种日期类型对应的最优路线。

In [ ]:
# 打印最优路线和行驶距离

print(f"最优行驶距离是：{10*round(m.objVal)} 英里。")

vals = m.getAttr('X', vars)
edges = selected(vals)

for k in dayType:
    tour = subtour(edges[k])
    tour.append(0) # 返回仓库
    print ("第 %i 天的路线: %s" % (k, " -> ".join(map(str, tour))))

## 参考文献

H. Paul Williams，《数学规划中的模型构建》，第五版。

版权所有 © 2020 Gurobi Optimization, LLC